<a href="https://colab.research.google.com/github/DariusBotusanu/RoFinBert/blob/main/fine_tuning_colab_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers datasets evaluate
! pip install accelerate -U

In [88]:
###IMPORTS###
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DistilBertTokenizer
import numpy as np
import evaluate

In [2]:
dataset = load_dataset("dariusbotusanu/ro_financial_phrasebank_v2")

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 3168
    })
    test_all_agree: Dataset({
        features: ['text', 'label'],
        num_rows: 680
    })
    test_75_agree: Dataset({
        features: ['text', 'label'],
        num_rows: 680
    })
})

In [95]:
tokenizer = DistilBertTokenizer.from_pretrained("racai/distilbert-base-romanian-cased", padding='max_length', max_length=45, truncation='max_length')

def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=45)

# Tokenize the datasets
tokenized_datasets = dataset.map(tokenize_function, batched=True)

#tokenized_train = dataset['train'].map(tokenization, batched=True)
#tokenized_eval = dataset['test_all_agree'].map(tokenization, batched=True)


Map:   0%|          | 0/3168 [00:00<?, ? examples/s]

Map:   0%|          | 0/680 [00:00<?, ? examples/s]

Map:   0%|          | 0/680 [00:00<?, ? examples/s]

In [97]:
tokenized_train = tokenized_datasets['train']
tokenized_eval = tokenized_datasets['test_all_agree']

In [101]:
tokenized_train

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 3168
})

In [104]:
model = AutoModelForSequenceClassification.from_pretrained("racai/distilbert-base-romanian-cased", num_labels=3)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at racai/distilbert-base-romanian-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [105]:
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    try:
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels)
    except Exception as e:
        print("Error at compute metrics")
        print(e)
        print(f"Logits shape: {logits.shape}")
        print(f"Labels shape: {labels.shape}")
        print(f"Predictions shape: {predictions.shape}")



training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [106]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    compute_metrics=compute_metrics,
)

c:\Users\Darius\miniconda3\envs\NLP_env\Lib\site-packages\accelerate\accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [107]:
trainer.train()

  0%|          | 0/1188 [00:00<?, ?it/s]

  0%|          | 0/85 [00:00<?, ?it/s]

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.